In [3]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models, datasets
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import KFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, precision_recall_fscore_support)
from transformers import ViTForImageClassification, ViTFeatureExtractor
import optuna
from torchvision.datasets import ImageFolder

In [4]:
dataset_dir ='/kaggle/input/deepfake/DeepFake'

In [5]:
# Transformations for training, validation, and testing
transform_train = transforms.Compose([
    transforms.Resize((299, 299)),  
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(15),
    transforms.RandomCrop(299, padding=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=20, scale=(0.8, 1.2), shear=10),
    transforms.RandomErasing(p=0.3),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_val_test = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [6]:
# Load the dataset
dataset = ImageFolder(root=dataset_dir, transform=transform_train)
print("Classes:", dataset.classes)
print("Class-to-Index Mapping:", dataset.class_to_idx)
print("Number of Samples:", len(dataset))

Classes: ['Fake', 'Real']
Class-to-Index Mapping: {'Fake': 0, 'Real': 1}
Number of Samples: 10826


In [12]:
# Function to get the Inception model (without fine-tuning)
def get_model(model_name):
    if model_name == "inception-v3":
        model = models.inception_v3(pretrained=True, aux_logits=True)

        # Freeze all layers
        for param in model.parameters():
            param.requires_grad = False

        # Modify the final fully connected layer for binary classification
        model.fc = nn.Linear(model.fc.in_features, 2) 

        return model

In [7]:
# Calculate metrics function
def calculate_metrics(model, loader, device):
    
    # Set the model to evaluation mode (disables dropout)
    model.eval()

    # Lists to store true labels and predicted labels
    all_labels = []
    all_predictions = []

    # Disabling gradient computation
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
             # Get predicted labels by taking the argmax (most likely class)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

     # Calculate the confusion matrix,which give TN, FP, FN, and TP
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    # Unpack the confusion matrix into four components: TN, FP, FN, TP
    TN, FP, FN, TP = conf_matrix.ravel() 

    total = conf_matrix.sum()
    accuracy = (TP + TN) / total if total > 0 else 0.0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return accuracy, precision, recall, f1, conf_matrix


In [8]:
# Train the model function with validation accuracy printed after each epoch
def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5):
    # Variable to track the best validation accuracy
    best_val_accuracy = 0
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
         # Iterate over batches in the training data
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)

            # If the output is a tuple (e.g., from Inception model), get the logits
            if isinstance(outputs, tuple):
                outputs = outputs[0]
                
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)

                 # If the output is a tuple (e.g., from Inception model), get the logits
                if isinstance(outputs, tuple):
                    outputs = outputs[0]
                    
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%")
        
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
    
    return best_val_accuracy

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cross-validation setup
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)


In [19]:
def objective(trial, model_name):
    # Get a suggested learning rate from Optuna
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    
    # Initialize the model with dropout
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    val_accuracies = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model and get validation accuracy
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on validation set
        val_accuracy, _, _, _, _ = calculate_metrics(model, val_loader, device)
        val_accuracies.append(val_accuracy)
    
    # Return the average validation accuracy across all folds as the objective value
    return np.mean(val_accuracies)


In [20]:
def evaluate_test_set(model_name, best_lr):
    # Initialize model with the best learning rate
    model = get_model(model_name).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    criterion = nn.CrossEntropyLoss()

    fold_metrics = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"\nEvaluating on Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on the test set
        test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
        fold_metrics.append(calculate_metrics(model, test_loader, device))
    
    # Print metrics for each fold
    for fold_idx, metrics in enumerate(fold_metrics):
        accuracy, precision, recall, f1, conf_matrix = metrics
        print(f"Fold {fold_idx + 1} Metrics:")
        print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")
        print(f"Confusion Matrix:\n{conf_matrix}")

# Calculate average metrics across folds
    avg_accuracy = np.mean([metrics[0] for metrics in fold_metrics])
    avg_precision = np.mean([metrics[1] for metrics in fold_metrics])
    avg_recall = np.mean([metrics[2] for metrics in fold_metrics])
    avg_f1 = np.mean([metrics[3] for metrics in fold_metrics])
    total_conf_matrix = np.sum([metrics[4] for metrics in fold_metrics], axis=0)

    print("\nAverage Metrics Across Folds:")
    print(f"Accuracy: {avg_accuracy:.2f}, Precision: {avg_precision:.2f}, Recall: {avg_recall:.2f}, F1-Score: {avg_f1:.2f}")
    print(f"Confusion Matrix (sum of all folds):\n{total_conf_matrix}")

In [21]:
for model_name in ["inception-v3"]:
    print(f"\nOptimizing for {model_name.upper()}...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, model_name), n_trials=5) 

    # Best learning rate found for the model
    best_lr = study.best_params['lr']
    print(f"Best Learning Rate for {model_name.upper()}: {best_lr}")

    # Evaluate on test sets for each fold
    evaluate_test_set(model_name, best_lr)

[I 2024-12-25 19:20:14,216] A new study created in memory with name: no-name-c0f869cb-61cf-4757-861b-c389146b4253



Optimizing for INCEPTION-V3...


<ipython-input-19-2e749100ab16>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)


Fold 1/3
Epoch 1/5, Loss: 2.795867316630664
Epoch 1/5, Validation Accuracy: 66.69%
Epoch 2/5, Loss: 3.2300137880757367
Epoch 2/5, Validation Accuracy: 56.99%
Epoch 3/5, Loss: 3.430133197189036
Epoch 3/5, Validation Accuracy: 66.48%
Epoch 4/5, Loss: 4.059932265492434
Epoch 4/5, Validation Accuracy: 54.85%
Epoch 5/5, Loss: 3.7087764647784156
Epoch 5/5, Validation Accuracy: 60.80%
Fold 2/3
Epoch 1/5, Loss: 3.5473217621692634
Epoch 1/5, Validation Accuracy: 65.79%
Epoch 2/5, Loss: 3.5817354818734017
Epoch 2/5, Validation Accuracy: 63.37%
Epoch 3/5, Loss: 4.040954607626351
Epoch 3/5, Validation Accuracy: 66.97%
Epoch 4/5, Loss: 3.677936596106429
Epoch 4/5, Validation Accuracy: 67.11%
Epoch 5/5, Loss: 3.6948164279948283
Epoch 5/5, Validation Accuracy: 55.96%
Fold 3/3
Epoch 1/5, Loss: 4.191949609234847
Epoch 1/5, Validation Accuracy: 67.66%
Epoch 2/5, Loss: 4.482568564994559
Epoch 2/5, Validation Accuracy: 64.47%
Epoch 3/5, Loss: 3.631193316443849
Epoch 3/5, Validation Accuracy: 66.90%
Epoch 

[I 2024-12-25 20:03:12,090] Trial 0 finished with value: 0.605724838411819 and parameters: {'lr': 0.04249100649730569}. Best is trial 0 with value: 0.605724838411819.
<ipython-input-19-2e749100ab16>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights

...

Fold 1/3
Epoch 1/5, Loss: 1.5928743642996688
Epoch 1/5, Validation Accuracy: 59.90%
Epoch 2/5, Loss: 1.4479668423615766
Epoch 2/5, Validation Accuracy: 62.81%
Epoch 3/5, Loss: 1.904955631132284
Epoch 3/5, Validation Accuracy: 56.99%
Epoch 4/5, Loss: 1.7519627690973862
Epoch 4/5, Validation Accuracy: 54.22%
Epoch 5/5, Loss: 1.5898829286928335
Epoch 5/5, Validation Accuracy: 65.93%
Fold 2/3
Epoch 1/5, Loss: 1.5696268252904904
Epoch 1/5, Validation Accuracy: 50.14%
Epoch 2/5, Loss: 1.8187278851619741
Epoch 2/5, Validation Accuracy: 63.43%
Epoch 3/5, Loss: 1.6177349829871353
Epoch 3/5, Validation Accuracy: 67.73%
Epoch 4/5, Loss: 1.9247395043873656
Epoch 4/5, Validation Accuracy: 52.29%
Epoch 5/5, Loss: 1.6977640375577283
Epoch 5/5, Validation Accuracy: 59.76%
Fold 3/3
Epoch 1/5, Loss: 1.6965398628738044
Epoch 1/5, Validation Accuracy: 53.74%
Epoch 2/5, Loss: 2.3102478121525674
Epoch 2/5, Validation Accuracy: 67.31%
Epoch 3/5, Loss: 1.7698275776199215
Epoch 3/5, Validation Accuracy: 63.85%

[I 2024-12-25 22:09:37,990] Trial 3 finished with value: 0.6435826408125577 and parameters: {'lr': 0.018597218691203208}. Best is trial 2 with value: 0.6793628808864266.
<ipython-input-19-2e749100ab16>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weig

Fold 1/3
Epoch 1/5, Loss: 0.7136827671066832
Epoch 1/5, Validation Accuracy: 49.03%
Epoch 2/5, Loss: 0.7057722158194906
Epoch 2/5, Validation Accuracy: 50.76%
Epoch 3/5, Loss: 0.7061050623161358
Epoch 3/5, Validation Accuracy: 54.29%
Epoch 4/5, Loss: 0.6980732695832437
Epoch 4/5, Validation Accuracy: 52.84%
Epoch 5/5, Loss: 0.6953908346634543
Epoch 5/5, Validation Accuracy: 55.26%
Fold 2/3
Epoch 1/5, Loss: 0.6909991947326871
Epoch 1/5, Validation Accuracy: 59.07%
Epoch 2/5, Loss: 0.6908998690257415
Epoch 2/5, Validation Accuracy: 61.50%
Epoch 3/5, Loss: 0.6844472983924065
Epoch 3/5, Validation Accuracy: 58.10%
Epoch 4/5, Loss: 0.6806260870965146
Epoch 4/5, Validation Accuracy: 61.15%
Epoch 5/5, Loss: 0.6813216667148948
Epoch 5/5, Validation Accuracy: 61.98%
Fold 3/3
Epoch 1/5, Loss: 0.6774347697832308
Epoch 1/5, Validation Accuracy: 63.57%
Epoch 2/5, Loss: 0.6723154977540284
Epoch 2/5, Validation Accuracy: 60.66%
Epoch 3/5, Loss: 0.6724251818261753
Epoch 3/5, Validation Accuracy: 63.78

[I 2024-12-25 22:51:14,567] Trial 4 finished with value: 0.6126500461680517 and parameters: {'lr': 1.1463590907212715e-05}. Best is trial 2 with value: 0.6793628808864266.
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Best Learning Rate for INCEPTION-V3: 0.0001153980122372495

Evaluating on Fold 1/3
Epoch 1/5, Loss: 0.6924630357415636
Epoch 1/5, Validation Accuracy: 62.33%
Epoch 2/5, Loss: 0.6629610618175064
Epoch 2/5, Validation Accuracy: 65.44%
Epoch 3/5, Loss: 0.6446246784694946
Epoch 3/5, Validation Accuracy: 65.37%
Epoch 4/5, Loss: 0.6363611804187627
Epoch 4/5, Validation Accuracy: 66.27%
Epoch 5/5, Loss: 0.6289965391817672
Epoch 5/5, Validation Accuracy: 66.90%

Evaluating on Fold 2/3
Epoch 1/5, Loss: 0.6257490301659094
Epoch 1/5, Validation Accuracy: 67.73%
Epoch 2/5, Loss: 0.6302250356963985
Epoch 2/5, Validation Accuracy: 68.91%
Epoch 3/5, Loss: 0.6138986237141308
Epoch 3/5, Validation Accuracy: 69.81%
Epoch 4/5, Loss: 0.6061086552577782
Epoch 4/5, Validation Accuracy: 68.42%
Epoch 5/5, Loss: 0.6091891309833
Epoch 5/5, Validation Accuracy: 68.63%

Evaluating on Fold 3/3
Epoch 1/5, Loss: 0.6087160082482501
Epoch 1/5, Validation Accuracy: 70.64%
Epoch 2/5, Loss: 0.6037638961280907
Epoch 2/5, 

In [9]:
def initialize_model(model_name):
    if model_name == "inception-v3":
        model = models.inception_v3(pretrained=True, aux_logits=True)  

        # Freeze all layers initially
        for param in model.parameters():
            param.requires_grad = False

        # Unfreeze the last two convolutional layers (Mixed_7b and Mixed_7c)
        for param in list(model.Mixed_7b.parameters()) + list(model.Mixed_7c.parameters()):
            param.requires_grad = True

        # Modify the final fully connected layer for binary classification
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, 2)  # 2 output classes

        # Print trainable parameters
        def count_trainable_params(model):
            return sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Total trainable parameters: {count_trainable_params(model):,}")

        return model
    else:
        raise ValueError("Model name must be 'inception-v3'")

In [10]:
# for fine tunning
def objective_after(trial, model_name):
    # Get a suggested learning rate from Optuna
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    
    # Initialize the model with dropout
    model = initialize_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    val_accuracies = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model and get validation accuracy
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on validation set
        val_accuracy, _, _, _, _ = calculate_metrics(model, val_loader, device)
        val_accuracies.append(val_accuracy)
    
    # Return the average validation accuracy across all folds as the objective value
    return np.mean(val_accuracies)

In [11]:
def evaluate_test(model_name, best_lr):
    # Initialize model with the best learning rate
    model = initialize_model(model_name).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    criterion = nn.CrossEntropyLoss()

    fold_metrics = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"\nEvaluating on Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on the test set
        test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
        fold_metrics.append(calculate_metrics(model, test_loader, device))
    
    # Print metrics for each fold
    for fold_idx, metrics in enumerate(fold_metrics):
        accuracy, precision, recall, f1, conf_matrix = metrics
        print(f"Fold {fold_idx + 1} Metrics:")
        print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")
        print(f"Confusion Matrix:\n{conf_matrix}")

    # Calculate average metrics across folds
    avg_accuracy = np.mean([metrics[0] for metrics in fold_metrics])
    avg_precision = np.mean([metrics[1] for metrics in fold_metrics])
    avg_recall = np.mean([metrics[2] for metrics in fold_metrics])
    avg_f1 = np.mean([metrics[3] for metrics in fold_metrics])
    total_conf_matrix = np.sum([metrics[4] for metrics in fold_metrics], axis=0)

    print("\nAverage Metrics Across Folds:")
    print(f"Accuracy: {avg_accuracy:.2f}, Precision: {avg_precision:.2f}, Recall: {avg_recall:.2f}, F1-Score: {avg_f1:.2f}")
    print(f"Confusion Matrix (sum of all folds):\n{total_conf_matrix}")

In [12]:
for model_name in ["inception-v3"]:
    print(f"\nOptimizing for {model_name.upper()}...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective_after(trial, model_name), n_trials=5)  # You can increase the number of trials if needed

    # Best learning rate found for the model
    best_lr = study.best_params['lr']
    print(f"Best Learning Rate for {model_name.upper()}: {best_lr}")

    # Evaluate on test sets for each fold
    evaluate_test(model_name, best_lr)

[I 2024-12-26 07:26:45,095] A new study created in memory with name: no-name-359ad20d-9a95-4576-a621-b7f5709cb1c2



Optimizing for INCEPTION-V3...


<ipython-input-10-fd9538108a4f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.

Total trainable parameters: 11,125,506
Fold 1/3
Epoch 1/5, Loss: 0.5638978836944749
Epoch 1/5, Validation Accuracy: 73.27%
Epoch 2/5, Loss: 0.4733071730611074
Epoch 2/5, Validation Accuracy: 77.15%
Epoch 3/5, Loss: 0.43890419017873417
Epoch 3/5, Validation Accuracy: 79.78%
Epoch 4/5, Loss: 0.40531234788960513
Epoch 4/5, Validation Accuracy: 77.77%
Epoch 5/5, Loss: 0.4050169918089282
Epoch 5/5, Validation Accuracy: 79.78%
Fold 2/3
Epoch 1/5, Loss: 0.4193113028015221
Epoch 1/5, Validation Accuracy: 83.45%
Epoch 2/5, Loss: 0.40484101485810886
Epoch 2/5, Validation Accuracy: 82.76%
Epoch 3/5, Loss: 0.37655094694037466
Epoch 3/5, Validation Accuracy: 83.24%
Epoch 4/5, Loss: 0.37620787338986583
Epoch 4/5, Validation Accuracy: 84.49%
Epoch 5/5, Loss: 0.36646624576320963
Epoch 5/5, Validation Accuracy: 82.20%
Fold 3/3
Epoch 1/5, Loss: 0.3744753759704242
Epoch 1/5, Validation Accuracy: 84.63%
Epoch 2/5, Loss: 0.3587072285005401
Epoch 2/5, Validation Accuracy: 83.59%
Epoch 3/5, Loss: 0.350180364

[I 2024-12-26 08:13:04,611] Trial 0 finished with value: 0.8277931671283473 and parameters: {'lr': 0.004118722506403129}. Best is trial 0 with value: 0.8277931671283473.
<ipython-input-10-fd9538108a4f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weig

...

Total trainable parameters: 11,125,506
Fold 1/3
Epoch 1/5, Loss: 0.5287911766800433
Epoch 1/5, Validation Accuracy: 72.85%
Epoch 2/5, Loss: 0.44426243649332564
Epoch 2/5, Validation Accuracy: 75.28%
Epoch 3/5, Loss: 0.4033638508115684
Epoch 3/5, Validation Accuracy: 80.06%
Epoch 4/5, Loss: 0.37710862355666924
Epoch 4/5, Validation Accuracy: 79.29%
Epoch 5/5, Loss: 0.3557982919789151
Epoch 5/5, Validation Accuracy: 82.06%
Fold 2/3
Epoch 1/5, Loss: 0.3875230983805261
Epoch 1/5, Validation Accuracy: 83.38%
Epoch 2/5, Loss: 0.3634129394648483
Epoch 2/5, Validation Accuracy: 84.90%
Epoch 3/5, Loss: 0.3433737968871607
Epoch 3/5, Validation Accuracy: 82.96%
Epoch 4/5, Loss: 0.33424263632758544
Epoch 4/5, Validation Accuracy: 84.35%
Epoch 5/5, Loss: 0.31496187427096606
Epoch 5/5, Validation Accuracy: 83.59%
Fold 3/3
Epoch 1/5, Loss: 0.34145854899237826
Epoch 1/5, Validation Accuracy: 85.73%
Epoch 2/5, Loss: 0.3263457348333538
Epoch 2/5, Validation Accuracy: 85.39%
Epoch 3/5, Loss: 0.3068726994

[I 2024-12-26 10:29:43,586] Trial 3 finished with value: 0.8252539242843951 and parameters: {'lr': 0.00017009810435342036}. Best is trial 2 with value: 0.8287165281625116.
<ipython-input-10-fd9538108a4f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `we

Total trainable parameters: 11,125,506
Fold 1/3
Epoch 1/5, Loss: 0.5446720842827749
Epoch 1/5, Validation Accuracy: 74.52%
Epoch 2/5, Loss: 0.4553913488901781
Epoch 2/5, Validation Accuracy: 79.16%
Epoch 3/5, Loss: 0.43898604653816853
Epoch 3/5, Validation Accuracy: 80.06%
Epoch 4/5, Loss: 0.4238336817335687
Epoch 4/5, Validation Accuracy: 78.95%
Epoch 5/5, Loss: 0.395289893390724
Epoch 5/5, Validation Accuracy: 79.50%
Fold 2/3
Epoch 1/5, Loss: 0.41691509086782763
Epoch 1/5, Validation Accuracy: 83.73%
Epoch 2/5, Loss: 0.4008506108384106
Epoch 2/5, Validation Accuracy: 83.66%
Epoch 3/5, Loss: 0.4060938256221582
Epoch 3/5, Validation Accuracy: 82.89%
Epoch 4/5, Loss: 0.3742943604512768
Epoch 4/5, Validation Accuracy: 82.96%
Epoch 5/5, Loss: 0.3718218248522743
Epoch 5/5, Validation Accuracy: 82.76%
Fold 3/3
Epoch 1/5, Loss: 0.3784575177490382
Epoch 1/5, Validation Accuracy: 83.38%
Epoch 2/5, Loss: 0.368230149849673
Epoch 2/5, Validation Accuracy: 83.73%
Epoch 3/5, Loss: 0.350881776839330

[I 2024-12-26 11:14:45,923] Trial 4 finished with value: 0.814173591874423 and parameters: {'lr': 0.0042578548751414326}. Best is trial 2 with value: 0.8287165281625116.
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Best Learning Rate for INCEPTION-V3: 0.000138700158394391
Total trainable parameters: 11,125,506

Evaluating on Fold 1/3
Epoch 1/5, Loss: 0.5376401125396812
Epoch 1/5, Validation Accuracy: 76.39%
Epoch 2/5, Loss: 0.44337052600818444
Epoch 2/5, Validation Accuracy: 77.84%
Epoch 3/5, Loss: 0.40468366024243896
Epoch 3/5, Validation Accuracy: 78.46%
Epoch 4/5, Loss: 0.3850684403055939
Epoch 4/5, Validation Accuracy: 78.67%
Epoch 5/5, Loss: 0.354561274881521
Epoch 5/5, Validation Accuracy: 81.09%

Evaluating on Fold 2/3
Epoch 1/5, Loss: 0.3820158474517791
Epoch 1/5, Validation Accuracy: 84.76%
Epoch 2/5, Loss: 0.36270403071661683
Epoch 2/5, Validation Accuracy: 83.24%
Epoch 3/5, Loss: 0.34414899662054704
Epoch 3/5, Validation Accuracy: 83.31%
Epoch 4/5, Loss: 0.32476432983717207
Epoch 4/5, Validation Accuracy: 85.04%
Epoch 5/5, Loss: 0.3107558508604271
Epoch 5/5, Validation Accuracy: 83.59%

Evaluating on Fold 3/3
Epoch 1/5, Loss: 0.34957296393029597
Epoch 1/5, Validation Accuracy: 85.39%
E